In [ ]:
!pip install openai

In [ ]:
!pip install mysql-connector-python

In [ ]:
# package

import openai
import mysql.connector
from pprint import pprint # chatGPT message가 잘 출력되되도록 사용
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

In [ ]:
# Google Drive 마운트
# 마운트는 본인 드라이브만 가능하기에 공유 폴더 같은 경우 본인 드라이브에 바로가기를 추가하여 사용

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# API KEY : contact 계정의 API key 발급하여 사용

openai.api_key = ""

In [ ]:
today = datetime.today().strftime("%Y%m%d")
Path(f"/content/drive/MyDrive/HAI research/output/{today}/py_output").mkdir(parents=True, exist_ok=True)

In [ ]:
# google sheet를 바로 연동하는 것은 원본 훼손 위험이 존재하기에 우선 xlsx 파일을 다운로드 및 업로드하여여 활용

raw_data = pd.read_excel("/content/drive/MyDrive/HAI research/data/PoC 평가.xlsx", sheet_name='total')

In [ ]:
en_data = raw_data.copy()

In [ ]:
# PoC 평가가 A+인 인원들만 불러오기
# url 주소에 userID가 존재하는 형식 데이터만 불러오기
# en_data = en_data.loc[en_data['총평 \n(Letter grade, like A+, B+)'] == 'A+']
en_data = en_data[en_data['framing docs'].str.contains('&userID', na = False)].reset_index(drop=True)

In [ ]:
# '&' 기준으로 파싱하여 user_id 리스트에 저장하기
id_list = []
for i in range(len(en_data)) :
  id_list.append(en_data['framing docs'][i].split('&')[1][7:])

In [ ]:
# DB 연동

mydb = mysql.connector.connect(
  host="",
  user="",
  password="",
  database=""
)

# 저장된 userID 정보 불러와서 사용
userCode = id_list[777]

mycursor = mydb.cursor()

mycursor.execute("SELECT user_id from userCodeTable where code='" + userCode + "'")

myresult = mycursor.fetchall()

userID = myresult[0][0]

mycursor.execute("SELECT user_id, question_id, response FROM responseTable where user_id='" + str(userID) + "'")

myresult = mycursor.fetchall()

userResponse = {}

# 0부터 시작되는 문항번호를 1부터 시작하도록 변경
for x in myresult:
    userResponse[x[1]+1] = x[2]

In [ ]:
userResponse

In [ ]:
id_list.index('on02m3H7')

In [ ]:
# 문항 정보 입력
qna = pd.read_csv("/content/drive/MyDrive/HAI research/data/PoC_Question.csv")

In [ ]:
objective = '시끄러운 퇴사자 예측'

In [ ]:
output = []

# userResponse[8] = objective

for i in range(10) :

  messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"},
              # {"role" : "system", "content" : f"너가 개선하고 싶은 프로세스는 {userResponse[5]}야"},
              {"role" : "system", "content" : "인간의 주관적인 감정과 경험은 인공지능을 사용하기에 적합하지 않은 영역이야"},
              {"role" : "system", "content" : "인공지능의 적합성 여부 이유를 설명할 때, 구체적으로, 단계적으로 설명해줘"},
              {"role" : "user", "content" : f"인공지능의 목적은 {userResponse[8]}이야. 해당 목적이 인공지능을 사용하기에 적합한지 예, 아니오로 답변해줘. 답변한 이후, 해당 이유를 설명해줘."}
              ]

  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=1,
  max_tokens=3000
  )

  chat_response = completion.choices[0].message.content
  # pprint(chat_response)

  output.append(chat_response)

In [ ]:
for i in range(10) :
  pprint(output[i])

In [ ]:
import json

In [ ]:
for i in range(len(output)) :
  output[i] = output[i].split('\n')

In [ ]:
file_path = f"/content/drive/MyDrive/HAI research/output/{today}/py_output/{userCode}.json"
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(json.dumps(output, ensure_ascii=False, indent=2))